# Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
from sklearn.preprocessing import MinMaxScaler,StandardScaler 
from sklearn.model_selection import train_test_split
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# Read data

In [4]:
sanimads = pd.read_csv("SANIMA_2000-01-01_2021-12-31.csv", engine='python')
sanimads.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
0,1,2021-12-29,169,40088.0,14203816.4,356.0,350.0,354.0
1,2,2021-12-28,225,36901.0,12875470.3,355.0,338.2,352.0
2,3,2021-12-27,311,63171.0,22589549.7,363.0,355.0,356.0
3,4,2021-12-26,312,53556.0,19125145.5,362.0,348.5,358.0
4,5,2021-12-23,267,37310.0,13066461.5,353.0,344.1,350.0


# Sort Data in Ascending order

In [5]:
sanimads = sanimads.sort_values(by="Date")
sanimads.head()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
2256,2257,2010-04-19,8,1025.0,405750.0,400.0,390.0,390.0
2255,2256,2010-04-20,12,1150.0,439525.0,385.0,379.0,384.0
2254,2255,2010-04-21,3,150.0,55850.0,377.0,370.0,370.0
2253,2254,2010-04-22,11,593.0,208378.0,365.0,342.0,359.0
2252,2253,2010-04-25,18,2825.0,969420.0,366.0,338.0,338.0


# Handle Missing NAs

In [6]:
#Handle missing data by dropinig NAs.
sanimads = sanimads.dropna()
sanimads.tail()

,S.N.,Date,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
4,5,2021-12-23,267,37310.0,13066461.5,353.0,344.1,350.0
3,4,2021-12-26,312,53556.0,19125145.5,362.0,348.5,358.0
2,3,2021-12-27,311,63171.0,22589549.7,363.0,355.0,356.0
1,2,2021-12-28,225,36901.0,12875470.3,355.0,338.2,352.0
0,1,2021-12-29,169,40088.0,14203816.4,356.0,350.0,354.0


# Plot Trend

In [7]:
sanimads.iplot(kind="line",x="Date",y="Close Price",xTitle="Date", yTitle="Close Price", title="SANIMA Trend")

# Next Price

In [8]:
sanimads['Next Price'] = sanimads.shift(-1)['Close Price']

In [9]:
sanimads= sanimads.dropna()


# Rename Columns

In [10]:
sanimads.rename(columns = {'S.N.':'SN','Total Transactions':'TTrans', 'Total Traded Shares':'TTS',
                              'Total Traded Amount':'TTA','Close Price':'ClosePrice','Next Price':'NextPrice',
                     'Max. Price':'MaxPrice','Min. Price':'MinPrice'}, inplace = True)

# Prepare Feature

In [11]:
features = ['ClosePrice',"Date"]
outFeature = ['NextPrice',"Date"]

In [12]:
# take close price as feature and next price as out feature
X, Y = sanimads[features], sanimads[outFeature] 
X.set_index("Date", inplace=True)
Y.set_index("Date", inplace=True)

# Normalize Data

In [13]:
#Normalize data using standard scalar.
ss = StandardScaler()

X["ClosePrice"] = ss.fit_transform(X)
Y["NextPrice"] = ss.fit_transform(Y)


## Train Test Split

Using train_test_split of sklearn will split data randomly so the sequence is lost.

In [14]:
def train_test_split(x,y,train_size):
    n = int(len(x)*train_size)
    trainx = x[:n]
    testx = x[n:]
    trainy=y[:n]
    testy=y[n:]
    return trainx,testx,trainy,testy

In [15]:
#Split train, validation, test set from X,Y to 80%, 10%, 10%

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9)

In [16]:
X_train.shape,X_test.shape,X.shape

((2030, 1), (226, 1), (2256, 1))

In [28]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(5,1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Convert to Matrix

In [29]:
# convert into dataset matrix
def convertToMatrix(data, step):
    data = data.to_numpy()
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

step=5

trainX,trainY =convertToMatrix(X_train,step)
testX,testY =convertToMatrix(X_test,step)
# validX,validY = convertToMatrix(X_valid,step)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1],1))
# validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1],1))

trainX.shape,testX.shape #,validX.shape

((2025, 5, 1), (221, 5, 1))

In [30]:
# #Reshape data for (Sample,Timestep,Features) 
# X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
# X_valid = X_valid.reshape((X_valid.shape[0],X_valid.shape[1],1))
# #print(X_train.shape)
# # print(X_valid.shape)


In [31]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(trainX, trainY, batch_size=1, epochs=1)

2025/2025 [==============================] - 10s 3ms/step - loss: 0.0461


In [49]:
predictions = model.predict(testX)
predictions = ss.inverse_transform(predictions)
predictions.shape,testX.shape

7/7 [==============================] - 0s 2ms/step


((221, 1), (221, 5, 1), (226, 1))

In [35]:
Y_test_ss = ss.inverse_transform(Y_test)[:-5]

rmse=np.sqrt(np.mean(((predictions- Y_test_ss)**2)))
print(rmse)

9.96499310800807


# Test Trend

In [50]:
test=pd.DataFrame()
test["pred"]=ss.inverse_transform(predictions.flatten())
test["real"]=ss.inverse_transform(Y_test_ss)
test["date"]=X_test.index[:-5]
test.iplot(kind="line", x="date")